In [1]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [2]:
pd.set_option('display.max_colwidth', 1000)

In [3]:
from tqdm import tqdm_notebook as tqdm

In [4]:
from nltk import word_tokenize


In [5]:
import re

In [59]:
from string import punctuation
punct = punctuation+'«»—…“”*№–\n'

In [7]:
stops = set(stopwords.words('russian'))

## Данные

In [8]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = './data'

In [9]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA) if not file.startswith('.')]

In [10]:
data = pd.concat([pd.read_json(file, lines=True) for file in files], axis=0, ignore_index=True)

In [11]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

Прежде чем приступать к экспериментам, я прогнала все подходы, испробованные на семинаре, на данных из Russia Today и обнаружила вот что: tfidf на этом датасете не добавило качества, а наоборот снизило ф-меру на 3%. Если подумать, это не то чтобы удивительно: высокий tfidf показывает, что слово часто встречается в тексте, но редко — во всей коллекции. Однако, например, слово “Россия” встречается в газетных текстах часто, 
поэтому его tfidf будет довольно низким — но оно наверняка будет ключевым словом почти для любого текста, в котором оно встречается. И наоборот: на RT был текст о том, как в Египте нашли какую-то мумию, — высокий tfidf там имело слово “останки”. Это естественно: “останки” редко встречаются в датасете, но постоянно встречались в этом тексте. Но ключевым словом они, конечно, не будут. <br>
<br>
Короче говоря, высокий tfidf не особо коррелирует с “ключёвостью” слова, поэтому считать его не надо, он может всё даже испортить, а если и повысил чуть-чуть качество, то это случайно так получилось.<br><br>

Я решила, что надо подумать, какие вообще слова попадают в ключевые, и просто посмотрела на них глазами. В ключевые слова попадают: <br>
именованные сущности <br>
аббревиатуры <br>
названия наук и всякие -измы и -ции <br>
коллокации типа “выборы президента” или “гражданская война” <br>
<br>
Также можно заметить, что в ключевые слова часто попадают слова, которые есть в заголовке или в саммари.<br>

Ещё есть такая проблема: лемматизация не всегда хороша, потому что иногда ключевые слова бывают во множественном числе. Можно попробовать брать слова и в единственном, и во множественном и надеяться, что полнота вырастет, ну а точность, ну что поделаешь.



Для начала попробуем выделить именованные сущности с помощью texterra.

In [12]:
import texterra

t = texterra.API('1ff8f795abee104d4eeb9fececde37addbeba255')


In [23]:
ex = "23 мая 2017 года Кирилл Серебренников пришел с обысками домой к режиссеру Кириллу Серебренникову и директору РАМТ Софье Апфельбаум (бывшая глава департамента государственной поддержки искусства Минкульта РФ и Кирилла Серебренникова)"


In [13]:
# берёт на вход строку и возвращает список именованных сущностей
def get_ne(text):
    nes = []
    entities = list(t.named_entities(text, rtype="entity", language="ru"))
    for entity in entities[0]:
        nes.append(entity[0])
    return nes
    
    

In [14]:
print(get_ne(ex))

['Кирилл Серебренников', 'Кириллу Серебренникову', 'Софье Апфельбаум', 'Минкульта РФ', 'Кирилла Серебренникова']


Надо лемматизировать.

In [14]:
def named_entities(text):
    ne_list = get_ne(text)
    tokens = [word_tokenize(ne) for ne in ne_list]
    new_ne_list = []
    for token in tokens:
        new = " ".join([morph.parse(word)[0].normal_form for word in token])
        new_ne_list.append(new)
    return list(set(new_ne_list))



In [16]:
print(named_entities(ex))

['софья апфельбаум', 'кирилл серебренников', 'кирилл серебренник', 'минкульт рф']


Плохие новости -- Текстерра работает очень долго. Попробуем всё же посмотреть на маленьком кусочке датасета, что получится, если лемматизировать и убрать повторы.

In [17]:
short_df = data.loc[:10]

In [18]:
%%time
short_df["named_entities"] = short_df["content"].apply(named_entities)

CPU times: user 378 ms, sys: 23.5 ms, total: 402 ms
Wall time: 1min 27s


/Users/ika/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
evaluate(short_df["keywords"], short_df["named_entities"])

Precision -  0.04
Recall -  0.23
F1 -  0.07
Jaccard -  0.04


In [20]:
short_df[["keywords", "named_entities"]]

,keywords,named_entities
0,"[школа, образовательные стандарты, литература, история, фгос]","[волков, ольга васильев, рф, российский федерация, сергей волков]"
1,"[красота, законы]",[]
2,"[юзефович, гражданская война, пепеляев, якутия]","[пермь, « казароз », кант, якутия, ссср, платон ойунский, ганнибал, якутский республика, якутск, ленский, леонид юзеф, сибирь, сасыл-сысый, стродиться, иван гончаров, охотский море, колчак, юзеф, тунгус, гражданский война, якут, огпу, советский, война, попов, унгерный, эрнест ренана, анатолий пепеляев, нина, леонид малышев, владимир короленко, иркутск, томск, москва, иван стродиться, пепеляев]"
3,"[формула1, автоспорт, гонки, испания, квят]","[барселона, льюис, кий райкконить, квёнок, пастор мальдонадо, даниил, red bull, гран-при испания, испания, honda, renault, кий райкконеный, дэн, ferrari льюис хэмилтон, марка уэббер, mercedes, вальттери боттас, гран-при это сезон, red bull racing, хельмут маркий, хэмилтон, райкконный, булья, даниил квить, себастьян феттель, кубок конструктор, алонсо, льюис хэмилтон, toro roso, фернандо алонсо, renault sport, сэр фрэнк, дженсон, лазурный берег, старое свет, williams, баттон, дженсон баттон, кий райкконный, mclaren, нико росберг, кий, тоть вольфф, маурицио арривабен, гран-при монако, ferrari, жак вильнева]"
4,"[есенин, православие, святая русь, поэзия, год литературы, клюев, мариенгоф, стихи, россия]","[россия, репин, юрий анненков, русь, ордынка, сергей, эрлихо, яков блюмкин, маяковский, « чёрный человек », август миклашевский, никитский бульвар, мариенгоф, микола чудотворец, сергей александр, ленинград, сереженька, мейерхольд, нью-йорк, анна изряднов, лев толстой, качалов, иван розанов, троцкий, борода, всеволод рождественский, лев николай, софья андрей, пушкин, софья андрей толстой-есенин, « есенин », сергей есенин, рапп, есенин, толстой, советский, левый давид, виктор мануйлов, василий наседкина, волошин, анатолий мариенгоф, северянин игорь, николай клюев, клычкова, бог, иисус, юпитер, европа, церковь, вольф эрлихо, питер, пятницкий, страстный монастырь, петра ганнушкин, зинаида райх, гпу, клюев, господь, москва, киров, екатерина, юрий либединский, миклашевский, августа миклашевский]"
5,"[медвузы, медицинское образование, рудн, николай стуров, интервью]","[российский университет дружба народ, подмосковье, тиаго фрейра да силва, бразилия, тиаго, москва, рудна]"
6,"[литература, книги, периодика, космос, небо, астрономия, анатомия, филология]","[россия, сеченов, московский медицинский академия имя . и.м ., урысон, мышей, ирина николай боголепов, м. : литтерра, эльвира гейдаровой, анна урысон, бакинский государственный университет, лал гулиев, саратовка, ленин, лебедев рана, яков перельман, лев этинген, баку, солнце, долгопрудный, метр ., анне владимир, закавказье, левый ефим этинген, лев лев колесников, московский государственный медико-стоматологический университет, эльвира гейдаров, азербайджан, физический институт имя . п.н, рана, земля, китай, николай i, львов]"
7,"[сша, ирак, война]","[джон абизайд, партизанский война, курдский, киркук, бадр, ирак, американец, сша, афганистан, иракский, хусейн, багдад, джордж буш, мосул, оон, война в ирак, эр-рияда, `` абрамс '']"
8,"[искусственный интеллект, робот, компьютер, технологии]","[провиденс, брауновский университет, углерод, одноклеточный, хлорофилл, белка, николай карамзин, дж . маккарти, dl, массачусетский технологический институт, ai, иван павлов, нью-джерси, иммануил кант, университет рутгерса, массачусетс]"
9,"[вб, вто, переговоры, тарифы]","[россия, кристалина георгиев, всемирный банка, джеймс вулфенсон, страна, рф, всемирный торговый организация ( вто, вб, полый вулфовице, вто, алексей гордеев, владимир ткаченко, российский]"


Полнота примерно на уровне бейзлайна, но точность очень низкая. В некоторых текстах извлекается слишком много именованных сущностей. Попробуем взять из них только самые частотные. 
Ещё есть проблема с лемматизацией -- получаются штука типа "российский федерация". Надо не приводить к нормальной форме, а только менять число и падеж, не трогая род.

In [15]:
def common_named_entities_2(text):
    ne_list = get_ne(text)
    tokens = [word_tokenize(ne.strip(punct)) for ne in ne_list]
    new_ne_list = []
    for token in tokens:
        norm = []
        for w in token:
            try:
                norm_w = morph.parse(w)[0].inflect({'sing', 'nomn'}).word
                norm.append(norm_w)
            except AttributeError:

                norm.append(w.lower())
        normalized = " ".join(norm)
        new_ne_list.append(normalized)
                
    common_nes = [x[0] for x in Counter(new_ne_list).most_common(5)]
    return common_nes



In [26]:
short_df["common_named_entities_2"] = short_df["content"].apply(common_named_entities_2)

/Users/ika/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
evaluate(short_df["keywords"], short_df["common_named_entities_2"])

Precision -  0.18
Recall -  0.18
F1 -  0.17
Jaccard -  0.12


Ура, мы побили бейзлайн! (ну, чуть-чуть)

In [29]:
short_df[["keywords", "common_named_entities_2"]]

,keywords,common_named_entities_2
0,"[школа, образовательные стандарты, литература, история, фгос]","[ольга васильев, рф, ольга васильева, волков, сергей волков]"
1,"[красота, законы]",[]
2,"[юзефович, гражданская война, пепеляев, якутия]","[пепеляев, юзефович, якутия, леонид юзефович, анатолий пепеляев]"
3,"[формула1, автоспорт, гонки, испания, квят]","[mclaren, ferrari, mercedes, renault, гран-при испания]"
4,"[есенин, православие, святая русь, поэзия, год литературы, клюев, мариенгоф, стихи, россия]","[есенин, клюев, мариенгоф, бог, советская]"
5,"[медвузы, медицинское образование, рудн, николай стуров, интервью]","[рудна, москва, подмосковье, бразилия, тиаго фрейра да силва]"
6,"[литература, книги, периодика, космос, небо, астрономия, анатомия, филология]","[азербайджан, россия, львов, анна урысон, закавказье]"
7,"[сша, ирак, война]","[ирак, американец, хусейн, джордж буш, война в ирак]"
8,"[искусственный интеллект, робот, компьютер, технологии]","[ai, dl, николай карамзин, иммануил кант, дж . маккарти]"
9,"[вб, вто, переговоры, тарифы]","[россия, кристалина георгиев, вто, вб, владимир ткаченко]"


Не забываем, что мы пока работаем с маленьким кусочком датасета. Как сработает лучший вариант с семинара на этом кусочке?

In [16]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [31]:
short_df['content_norm'] = short_df['content'].apply(normalize)

/Users/ika/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
evaluate(short_df['keywords'], short_df['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.12
Recall -  0.27
F1 -  0.16
Jaccard -  0.09


Ну да, ф-меру мы улучшили на 1%

Кстати, сколько в среднем ключевых слов в тексте? Это нам потом когда-нибудь пригодится.

In [33]:
number_of_keywords = [len(lst) for lst in data["keywords"]]
mean_number_of_keywords = np.mean(number_of_keywords)
mean_number_of_keywords

6.1177654755913435

Теперь попробуем вытащить из текстов аббревиатуры (некоторые уже достаются текстеррой, но не все).

In [17]:
def get_abbrs(text):
    abbrs = []
    for word in word_tokenize(text):
        word = word.strip(punct)
        if word.isupper() and len(word) > 1:
            abbrs.append(word.lower())
    return list(set(abbrs))
    

In [35]:
short_df['abbreviations'] = short_df['content'].apply(get_abbrs)

/Users/ika/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [36]:
evaluate(short_df["keywords"], short_df["abbreviations"])

Precision -  0.2
Recall -  0.11
F1 -  0.13
Jaccard -  0.08


Одни аббревиатуры уже неплохо зашли. Сейчас мы их ещё совместим с именованными сущностями.

In [37]:
short_df[["keywords", "common_named_entities_2", "abbreviations"]]

,keywords,common_named_entities_2,abbreviations
0,"[школа, образовательные стандарты, литература, история, фгос]","[ольга васильев, рф, ольга васильева, волков, сергей волков]","[рф, нг, фгос]"
1,"[красота, законы]",[],[]
2,"[юзефович, гражданская война, пепеляев, якутия]","[пепеляев, юзефович, якутия, леонид юзефович, анатолий пепеляев]","[огпу, ссср, цик]"
3,"[формула1, автоспорт, гонки, испания, квят]","[mclaren, ferrari, mercedes, renault, гран-при испания]",[]
4,"[есенин, православие, святая русь, поэзия, год литературы, клюев, мариенгоф, стихи, россия]","[есенин, клюев, мариенгоф, бог, советская]",[гпу]
5,"[медвузы, медицинское образование, рудн, николай стуров, интервью]","[рудна, москва, подмосковье, бразилия, тиаго фрейра да силва]",[рудн]
6,"[литература, книги, периодика, космос, небо, астрономия, анатомия, филология]","[азербайджан, россия, львов, анна урысон, закавказье]","[авропа, ран, xx, xix, xi–xiv, и.м, xxi, л.г, в.и, хх, iv, нгн, п.н]"
7,"[сша, ирак, война]","[ирак, американец, хусейн, джордж буш, война в ирак]","[оон, едва, сша]"
8,"[искусственный интеллект, робот, компьютер, технологии]","[ai, dl, николай карамзин, иммануил кант, дж . маккарти]","[dl, amt, ai]"
9,"[вб, вто, переговоры, тарифы]","[россия, кристалина георгиев, вто, вб, владимир ткаченко]","[рф, ввп, вб, вто]"


Сделаем так: сольём в один список все именованные сущности и аббревиатуры (аббревиатуры могут повторяться, ну что поделаешь), а потом оттуда возьмём 6 самых частотных слов.<br>
Надо будет немного поменять функции.


In [18]:
def named_entities_2(text):
    ne_list = get_ne(text)
    tokens = [word_tokenize(ne.strip(punct)) for ne in ne_list]
    new_ne_list = []
    for token in tokens:
        norm = []
        for w in token:
            try:
                norm_w = morph.parse(w)[0].inflect({'sing', 'nomn'}).word
                norm.append(norm_w)
            except AttributeError:

                norm.append(w.lower())
        normalized = " ".join(norm)
        new_ne_list.append(normalized)
    return new_ne_list

In [19]:
def get_all_abbrs(text):
    abbrs = []
    for word in word_tokenize(text):
        word = word.strip(punct)
        if word.isupper() and len(word) > 1:
            abbrs.append(word.lower())
    return abbrs

In [20]:
def nes_and_abbrs(text):
    nes = named_entities_2(text)
    abbrs = get_all_abbrs(text)
    together = nes + abbrs
    return [x[0] for x in Counter(together).most_common(6)]
    

In [42]:
short_df['nes_and_abbrs'] = short_df['content'].apply(nes_and_abbrs)

/Users/ika/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [43]:
evaluate(short_df["keywords"], short_df["nes_and_abbrs"])

Precision -  0.2
Recall -  0.25
F1 -  0.21
Jaccard -  0.13


Вот, стало ещё получше.

In [44]:
short_df[["keywords", "nes_and_abbrs"]]

,keywords,nes_and_abbrs
0,"[школа, образовательные стандарты, литература, история, фгос]","[рф, ольга васильев, нг, фгос, ольга васильева, волков]"
1,"[красота, законы]",[]
2,"[юзефович, гражданская война, пепеляев, якутия]","[пепеляев, юзефович, якутия, леонид юзефович, анатолий пепеляев, ссср]"
3,"[формула1, автоспорт, гонки, испания, квят]","[mclaren, ferrari, mercedes, renault, гран-при испания, барселона]"
4,"[есенин, православие, святая русь, поэзия, год литературы, клюев, мариенгоф, стихи, россия]","[есенин, клюев, мариенгоф, бог, гпу, советская]"
5,"[медвузы, медицинское образование, рудн, николай стуров, интервью]","[рудна, рудн, москва, подмосковье, бразилия, тиаго фрейра да силва]"
6,"[литература, книги, периодика, космос, небо, астрономия, анатомия, филология]","[азербайджан, россия, львов, ран, анна урысон, закавказье]"
7,"[сша, ирак, война]","[ирак, оон, американец, хусейн, сша, джордж буш]"
8,"[искусственный интеллект, робот, компьютер, технологии]","[ai, dl, николай карамзин, иммануил кант, дж . маккарти, университет рутгерса]"
9,"[вб, вто, переговоры, тарифы]","[вто, россия, вб, кристалина георгиев, рф, владимир ткаченко]"


Возьмём другой слайс датафрейма.

In [23]:
little_df = data.loc[1000:1010].reset_index()


In [46]:
little_df['nes_and_abbrs'] = little_df['content'].apply(nes_and_abbrs)

In [47]:
evaluate(little_df["keywords"], little_df["nes_and_abbrs"])

Precision -  0.14
Recall -  0.13
F1 -  0.13
Jaccard -  0.08


А вот на этом кусочке датасета наш метод работает хуже -- зависит от куска, тут как повезёт. Хорошо бы проверить на всём датасете, но очень уж долго работает Текстерра. Потом возьму кусок побольше.

Попробуем ещё добавить слова на -логия, -изм, -ция. 

In [20]:
def get_isms(text):
    norm_text = [morph.parse(w)[0].normal_form for w in word_tokenize(text)]
    isms = [w for w in norm_text if w.endswith("логия") or w.endswith("ция") or w.endswith("изм")]
    return isms

Попробуем совместить с ИС и аббревиатурами.

In [21]:
def nes_abbrs_isms(text):
    nes = named_entities_2(text)
    abbrs = get_all_abbrs(text)
    isms = get_isms(text)
    together = nes + abbrs + isms
    return [x[0] for x in Counter(together).most_common(6)]

Вообще не очень, конечно, хорошо получается: три раза проходишь каждый текст в поисках разных слов. Но это всё оттого, что мы добавляем искомые категории слов по одной. По-хорошему, надо потом сделать аккуратно, чтобы пройти текст один раз и сразу достать всё, что нужно, и чтобы код не повторялся.

In [24]:
little_df['nes_abbrs_isms'] = little_df['content'].apply(nes_abbrs_isms)

In [25]:
evaluate(little_df["keywords"], little_df["nes_abbrs_isms"])

Precision -  0.15
Recall -  0.15
F1 -  0.14
Jaccard -  0.09


In [26]:
little_df[["keywords", "nes_abbrs_isms"]]

,keywords,nes_abbrs_isms
0,"[театр ленсовета, юрий бутусов, мид, британский совет, фонд кино, вциом, кинотеатры, телевидение, троцкий]","[москва, юрий бутусов, театр ленсовет, константин сухенко, бутусов-чиновник, андрей миронов]"
1,"[фестиваль, утро родины, миронов]","[революция, ленин, миронов, сахалин, троцкий, южно-сахалинск]"
2,"[проза, женщины, международный женский день, иностранные языки, гендер, стихи, любовь, песни, президент, стюардессы, весна]","[виктория балашов, елена фёдоров, александр чистяков, фёдоров, балашов, фёдорова]"
3,"[сирия, вооруженный конфликт, восточная гута, гуманитарная пауза, резолюция, россия, франция]","[франция, оон, резолюция, макрон, сб, сирийское]"
4,"[минюст, запрос, кс, еспч, юкос, компенсации]","[кс, еспч, рф, юкос, конституция, министерство юстиция рф]"
5,"[электромобиль, экология, бензин, виэ]","[кобальт, россия, сша, китай, рф, конго]"
6,"[оскар, кинопремия, кино, киммел, форма воды]","[номинация, дискриминация, джимми киммел, грета гервиг, дель торо, макдорманд]"
7,"[автоспорт, туринг, гонки, wtcc, германия, норжшляйфе, северная петля, нюрбургринг]","[wtcc, сабина, fia, тс1, чемпионат мир, fia wtcc]"
8,"[поэзия, дом брюсова, музей серебряного века, презентация, франция, переводы, осип мандельштам, михаил шемякин, парижский книжный салон, солженицын]","[франция, россия, валерий нарбикова, александр солженицын, париж, французский язык]"
9,"[сирия, восточная гута, котел, рассекать, инициатива, военное планирование, генштаб]","[сирия, дамаск, москва, вашингтон, сдс, турецкий]"


Получается не очень. Может, из-за того, что текстов мало, а может, просто плохая идея доставать -измы.

А попробуем достать типа именованные сущности без текстерры, чтобы всё-таки пройти весь датасет, а не по кусочкам. Здесь же аббревиатуры.

In [55]:
def sorta_ne(text):
    tokens = word_tokenize(text)
    abbrs = []
    nes = []
    for i in range(len(tokens)):
        if (len(tokens[i]) > 1):
            if tokens[i].isupper():
                abbrs.append(tokens[i])
            else:
                if tokens[i][0].isupper()\
                and (not tokens[i-1].endswith((".", "!", "?", "—", "«", "…", "“", "\"", "\n"))):
                    nes.append(tokens[i])
    result = abbrs + nes
    return result

In [58]:
def named_entities(text):
    ne_list = sorta_ne(text)
    nes = [morph.parse(w)[0] for w in ne_list]
    nes = [w.normal_form for w in nes if w.tag.POS == 'NOUN']
    return [x[0] for x in Counter(nes).most_common(6)]

In [60]:
%%time
data["named_entities"] = data["content"].apply(named_entities)

CPU times: user 18.8 s, sys: 41.2 ms, total: 18.8 s
Wall time: 18.9 s


In [61]:
evaluate(data["keywords"], data["named_entities"])

Precision -  0.17
Recall -  0.19
F1 -  0.17
Jaccard -  0.1


Ну, чуть-чуть лучше, чем бейзлайн. Конечно, текстерра находит именованные сущности лучше.

In [63]:
def isms(text):
    norm_text = [morph.parse(w)[0].normal_form for w in word_tokenize(text)]
    isms = [w for w in norm_text if w.endswith("логия") or w.endswith("ция") or w.endswith("изм")]
    return isms

In [67]:
def all_named_entities(text):
    ne_list = sorta_ne(text)
    nes = [morph.parse(w)[0] for w in ne_list]
    nes = [w.normal_form for w in nes if w.tag.POS == 'NOUN']
    
    return nes

In [68]:
def nes_and_isms(text):
    nes = all_named_entities(text)
    ism = isms(text)
    together = nes + ism
    return [x[0] for x in Counter(together).most_common(6)]

In [69]:
%%time
data["nes_and_isms"] = data["content"].apply(nes_and_isms)

CPU times: user 1min 14s, sys: 101 ms, total: 1min 14s
Wall time: 1min 14s


In [70]:
evaluate(data["keywords"], data["nes_and_isms"])

Precision -  0.18
Recall -  0.2
F1 -  0.18
Jaccard -  0.11


Ну, это больше 16%.<br>

Теперь возьмём кусок датасета побольше и попробуем на нём только именованные сущности и аббревиатуры.

In [76]:
bigger_df = data.loc[::4].reset_index()

In [ ]:
evaluate(bigger_df["keywords"], bigger_df["nes_and_abbrs"])

Ииииииии оно два раза не досчиталось, и больше времени нет. Вообще интересно, конечно, что там было бы. Потом попробую запустить.

## Итоги

Больше я ничего не успеваю, поэтому будем подводить итоги. 

**Именованные сущности и аббревиатуры** часто попадают в ключевые слова, они повышают качество. <br>

Со **словами на -логия, -изм, -ция** непонятно: скорее всего, они сильно не повысят качество, а ещё там извлекаются совсем неподходящие лишние слова.


## Что ещё я собиралась сделать, но не успела

1. Иногда извлечённые именованные сущности не совпадают с эталонными, потому что в эталонных только фамилия человека, а у нас вместе с именем (а бывает и наоборот). В текстерре можно посмотреть, к какому типу относится именованная сущность: если это персона, то можно брать только фамилию -- тогда качество может повыситься. (А вообще, если бы мы мерили качество как-то по-другому, а не по имеющемуся списку, то это было бы неважно: какая разница, ключевое слово "солженицын" или "александр солженицын", и то и другое одинаково хорошо. То же касается случаев "россия" и "рф").

2. Хорошо было бы попробовать извлечь частотные коллокации типа "выборы президента". 

3. Ещё, я думаю, помогли бы частотные слова из статьи, которые также встречаются в заголовке и в саммари текста (особенно с саммари хорошо бы получилось).



## Попробуем другую штуку для именованных сущностей!

In [31]:
import spacy

spacy = spacy.load('xx')

In [41]:
def ner(text):
    entities = spacy(text).ents
    result = [str(entity) for entity in entities]
    return result

In [48]:
print(ner(ex))

['Кирилл Серебренников', 'Кириллу Серебренникову', 'РАМТ Софье Апфельбаум', 'Минкульта РФ', 'Кирилла Серебренникова']


In [72]:
def named_entities_3(text):
    ne_list = ner(text)
    tokens = [word_tokenize(ne.strip(punct)) for ne in ne_list]
    new_ne_list = []
    for token in tokens:
        norm = []
        for w in token:
            try:
                norm_w = morph.parse(w)[0].inflect({'sing', 'nomn'}).word
                norm.append(norm_w)
            except AttributeError:

                norm.append(w.lower())
        normalized = " ".join(norm)
        new_ne_list.append(normalized)
    return [ent for ent in new_ne_list if ent]

In [46]:
named_entities_3(ex)

['кирилл серебренник',
 'кирилл серебренников',
 'рамт софья апфельбаум',
 'минкульт рф',
 'кирилл серебренников']

In [73]:
def nes_and_abbrs_2(text):
    nes = named_entities_3(text)
    abbrs = get_all_abbrs(text)
    together = nes + abbrs
    return [x[0] for x in Counter(together).most_common(6)]
    

In [50]:
%%time
data['nes_and_abbrs_2'] = data['content'].apply(nes_and_abbrs_2)

CPU times: user 5min 36s, sys: 49.2 s, total: 6min 25s
Wall time: 3min 23s


In [51]:
evaluate(data["keywords"], data["nes_and_abbrs_2"])

Precision -  0.15
Recall -  0.17
F1 -  0.15
Jaccard -  0.09


Ахаха, что-то это хуже ожидаемого!

In [52]:
data[["keywords", "nes_and_abbrs_2"]]

,keywords,nes_and_abbrs_2
0,"[школа, образовательные стандарты, литература, история, фгос]","[, нг, фгос, фгоса, рф, совет]"
1,"[красота, законы]","[, если, вам, хорошо, вряд, где]"
2,"[юзефович, гражданская война, пепеляев, якутия]","[, пепеляев, юзефович, якутия, но, в]"
3,"[формула1, автоспорт, гонки, испания, квят]","[mercedes, ferrari, mclaren, williams, renault, ]"
4,"[есенин, православие, святая русь, поэзия, год литературы, клюев, мариенгоф, стихи, россия]","[есенин, , клюев, но, я, гпу]"
5,"[медвузы, медицинское образование, рудн, николай стуров, интервью]","[, рудн, начавший, считающий, мы понимающий, наш выпускник]"
6,"[литература, книги, периодика, космос, небо, астрономия, анатомия, филология]","[, азербайджан, вселенная, что, лексикон, впрочем]"
7,"[сша, ирак, война]","[ирак, , едва, хусейн, сша, джордж буш]"
8,"[искусственный интеллект, робот, компьютер, технологии]","[ai, , dl, но, николай карамзин, письмо]"
9,"[вб, вто, переговоры, тарифы]","[вто, россия, вб, кристалина георгиев, , организация]"


Какие именованные сущности странные. Подкрутим чуть-чуть

In [68]:
%%time
data['nes'] = data['content'].apply(named_entities_3)

CPU times: user 5min 13s, sys: 48.8 s, total: 6min 2s
Wall time: 3min 8s


In [70]:
data[["keywords", "nes"]].head()

,keywords,nes
0,"[школа, образовательные стандарты, литература, история, фгос]","[совет, фгос, министерство, новое фгоса, на, фгос, фгосама, напомнивший, гильдия, рф ольга васильева . по мнение, министерство, однако, накануне, совет, фгосама, ольга васильев прокомментировавшая, гильдия, волков, facebook, волков, ольга васильев . – вспомнивший, стих, следующий, нг, сохранённая, а весь, фгоса, ольга васильев ., фгоса, ольга васильев, в, способность, в, сформированность, кроме то, ольга васильев, для, почему, после, что, фгоса, проблема]"
1,"[красота, законы]","[хорошо, плохо, слава, вряд, возможно . а, уходящий, почему, где, предмет, где, если, она . пассия, одна, можно, пожалуй . когда, сначала, вряд, во-вторых, год, красота . когда, жаль, для, пришедшийся, или, на, однако, переживший, второй, нельзя, следующий, если, хорошо, нет-нувший, если, заигравшийся . что, отнюдь . учащий, далёкий, осторожный, знающий, большой, означающий, или, спокойствие . третий, только, оно, вас, наступающий, вставший, а красота, понявший, перед, после, первый, ваби-саби, для, если, понявший, отсозерцавший, наставшее, большая, она, вам, вам, вам, вкус]"
2,"[юзефович, гражданская война, пепеляев, якутия]","[юзефович, унгерная, в, якутия, анатолий пепеляев . и, тема, ведь, пепеляев, пепеляев оставшийся, этот, однако, пепеляев, для, колчак, юзефович « казароз, здесь, гражданская, начальник чукотка, белое, тринадцатый, юзефович, герой, пепеляев, москва, совет, якутия, сам пепеляев, эрнест ренана, даже, нина, восточное, пепеляев, офицерский, томск . он, леонид малышев, ленский, кант, председатель, республика, платон, иван стродившийся, сибирь, юзефович . в место, иван гончаров, владимир короленко, там, юзефович, в, зимой по ночь, условие . при, маленькая, кульминация, крепостная, впоследствии стродившийся, сегодня, под, юзефович, один, сасыл-сысый, но, в якутск, ганнибал, якутия, охотское, пепеляев, восставший, иркутск, но, сибирь . против, пепеляев, в якутия, сам анатолий пепеляев, огпу, якутия, сибирь, но]"
3,"[формула1, автоспорт, гонки, испания, квят]","[гран-при испания, формулы-1, барселона, старое свет, победный, mercedes, нико росберг . себастьян феттель, ferrari, мерседес, вторая . кий райкконный, williams, ferrari, mclaren, manor, если, пастор мальдонадо, фернандо алонсо, формулы-1, гран-при испания, время . кое-какая, также, второе, себастьян феттель, льюис хэмилтон, хэмилтон, льюис, далее, williams, кий райкконеный, ferrari, кий финишировавший аккурат, перед, уильямс, ferrari, так и произошедшее . сюрприз, renault, mclaren, в, даниил квивший, жак вильнева, хельмут маркое, квёнок, сегодня даниил, кроме, red bull, дэн, кстати, даниил, тут, даниил, марка уэббер, renault, гран-при испания . глава renault sport, toro roso, renault, после, новинка, барселона, себастьян феттель, кий райкконный, версия, судящий, мы сделавший небольшой, маурицио, арривабен . о то, ferrari, кий райкконённая, на этот раз кий, сегодня, mercedes, вероятно, барселона, гран-при, уик-энд . мы, сегодня, сэр фрэнк, уик-энд . подиум, williams, начавшее, разр..."
4,"[есенин, православие, святая русь, поэзия, год литературы, клюев, мариенгоф, стихи, россия]","[есенин, гпу, она, есенин, непосредственное, яков блюмкин, троцкий . а, левый давидович, есенин, самое, есенин, но, всеволод, рождественский, виктор мануйлов : « умерший сергей есенин . убивший, есенин, москва . уже, он уже ходивший, остановившийся, но, василий наседкина, есенин, екатерина . – передо, надоевшее весь, анна изряднов, есенин, сматывающийся, выдающийся, юрий анненков, есенин, путь, в, год, россия, хорошо, в страна, можно, только, маяковский . юбилейное . моя, большой, да, сам я тоже, есенин . русь, дело, незадолго, есенин, ленинград, киров, но, августа миклашевская, любовь, ленинград, женившийся . новая, софья андреевна толстая-есенина, волошин, весь, хотящий, страдавший, есенин, писатель юрий либединский, есенин, рапп, я, но не помнящий, скучно . борода, какая, то, раз, лев николаевич, толстой вместе, лев, никол

А что там вообще spacy выделяет-то?

In [56]:
data['ents'] = data["content"].apply(ner)

In [77]:
data[["keywords", "ents"]].head()

,keywords,ents
0,"[школа, образовательные стандарты, литература, история, фгос]","[Совета, ФГОС, Министерстве, Новые ФГОСы, \nНа, ФГОС, ФГОСам, \nНапомним, Гильдия, РФ Ольги Васильевой. По мнению, Министерство, –, Однако, \nНакануне, Совета, ФГОСам, Ольга Васильева прокомментировала, Гильдии, Волков, Facebook, Волков., –, Ольга Васильева. – Вспомните, –, Стихи –, Следующий, «НГ, Сохранена, А все, ФГОСа, –, Ольга Васильева. –, ФГОСа, Ольга Васильева, В, Способность, \nВ, Сформированность, \n«Кроме того, –, Ольга Васильева, –, Для, Почему, \nПосле, –, –, Что, ФГОСа, Проблема]"
1,"[красота, законы]","[Хорошо, Плохо, Слава, вряд, возможно. \nА, уходит., Почему, , –, Где, –, Предмет, –, Где, \nЕсли, –, она. Пассия, Одна, –, \nМожно, –, Пожалуй. Когда, –, –, Сначала \n, вряд, Во-вторых, лет., Красота. Когда, –, \nЖаль, –, , \nДля, Придется, Или, \nНа, Однако, Переживите, Второй, Нельзя, \nСледующим, Если, , Хорошо, Нет-нет, Если, –, –, Заигрались. Что, Отнюдь. Учите, \n Дальше, Осторожней, –, Знайте, Больше, Означает, Или –, –, \nСпокойствие. Третий, Только, Оно, вас., Наступает, Встаньте, –, А красота, – , Поняли, –, \nПеред, После, Первым, ваби-саби., Для, Если, Понять, –, \nОтсозерцали, –, Настало, –, Большую, , –, Она, вам, вам, , вам, вкусу.]"
2,"[юзефович, гражданская война, пепеляев, якутия]","[Юзефович, Унгерна, –, –, В, Якутии, Анатолия Пепеляева. И, Тема, Ведь, Пепеляев, Пепеляева остался, Этот, Однако, Пепеляев, Для, Колчака, Юзефовича «Казароза, Здесь, Гражданская, –, Начальник Чукотки, Белое, Тринадцать, Юзефович, Герой, Пепеляев, Москвы, Советами., Якутии, Сам Пепеляев, Эрнеста Ренана, Даже, Ниной, –, Восточного, Пепеляев, Офицерского, Томске. Его, Леонид Малышев, Ленский, Канта, Председателем, Республики, Платон, Иван Строд, Сибири., Юзефович. В местах, Иван Гончаров, Владимир Короленко, \nТам, Юзефович, В, Зимой по ночам, условиях. При, Маленькая, Кульминацией, Крепостную, Впоследствии Строд, Сегодня, \nПод, Юзефовича, Один, –, Сасыл-Сысы, Но, \nВ Якутске, Ганнибал, Якутии, –, Охотского, Пепеляеве, Восставшие, Иркутск, Но, Сибири. Против, Пепеляева, в Якутии, Сам Анатолий Пепеляев, ОГПУ, Якутии, Сибири, Но]"
3,"[формула1, автоспорт, гонки, испания, квят]","[Гран-При Испании, Формулы-1, Барселоны, Старого Света., Победный, Mercedes, Нико Росбергу. Себастьян Феттель, Ferrari, «Мерседесы», вторую. Кими Райкконен, Williams, Ferrari, McLaren, Manor, \nЕсли, Пастора Мальдонадо, Фернандо Алонсо, Формулы-1, Гран-При Испании, время. Кое-какая, Также, второе, Себастьяном Феттелем, Льюисом Хэмилтоном, Хэмилтона, Льюис, Далее, Williams, Кими Райкконеном, Ferrari, Кими финишировал аккурат, Перед, Уильямсы, Ferrari, Так и произошло. Сюрпризов, Renault, McLaren, \n В, Даниил Квят, Жака Вильнева, Хельмута Марко, Квята, Сегодня Даниил, Кроме, Red Bull, Дэна, Кстати, Даниила, Тут, Даниила, Марка Уэббера, Renault, Гран-При Испании. Глава Renault Sport, Toro Roso, Renault, После, \n Новинки, Барселону, Себастьян Феттель, Кими Райкконен –, версии., Судя, Мы сделали небольшой, Маурицио, Арривабене. О том, Ferrari, Кими Райкконена, На этот раз Кими, Сегодня, Mercedes, Вероятно, Барселоне, Гран-При, уик-энд. Мы, Сегодня, Сэра Фрэнка, уик-эндом. Подиум, Will..."
4,"[есенин, православие, святая русь, поэзия, год литературы, клюев, мариенгоф, стихи, россия]","[Есенина, ГПУ, Она, Есенин, Непосредственным, Якова Блюмкина, Троцкого. А, Лев Давидович, Есенин, \nСамое, –, Есенина, Но, Всеволод, Рождественский, Виктору Мануйлову: «Умер Сергей Есенин. Убил, Есенина, Москве. Уже, Он уже ходил, Остановившиеся, Но, –, Василий Наседкин, Есенина, Екатерины. – Передо, Надоело все», Анну Изряднову, Есенин, Сматываюсь, Выдающийся, Юрий Анненков, Есенин, –, Пути, В, году, Россия, \nХорошо, \nв Стране, Можно, \nТолько, –, \nМаяковский. Юбилейное.\nМоя, больше \n, \nДа, \nсам я тоже, Есенин. Русь, \nДело, Незадолго, Есенин, Ленинград, Кирова, Но, –, Августой Миклашевской, Любовь, Ленинград., –, –, женился. Новой, Софья Андреевна Толстая-Есенина, Волошиных, Все,

Да уж, текстерра получше выделяла. Запустим ещё раз, с небольшой очисткой.

In [74]:
%%time
data['nes_and_abbrs_2'] = data['content'].apply(nes_and_abbrs_2)

CPU times: user 5min 35s, sys: 49.5 s, total: 6min 24s
Wall time: 3min 22s


In [75]:
evaluate(data["keywords"], data["nes_and_abbrs_2"])

Precision -  0.16
Recall -  0.17
F1 -  0.16
Jaccard -  0.09


В общем, spacy выделяет так себе, поэтому проверить гипотезу про именованные сущности на всём корпусе всё-таки не удалось.